In [34]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pythainlp.tokenize import word_tokenize
from pythainlp.util import normalize
import os

In [35]:
import os

# ตั้งค่าให้ Ollama ใช้เซิร์ฟเวอร์ที่รันอยู่บนเครื่องอื่น
os.environ["OLLAMA_BASE_URL"] = "http://ollama:11434"

In [36]:
OLLAMA_BASE_URL = "http://ollama:11434"
OLLAMA_LLM_MODEL = "deepseek-r1:8b"
OLLAMA_EMBED_MODEL = "nomic-embed-text"

In [37]:
def read_thai_pdfs(pdf_directory):
    """
    อ่านไฟล์ PDF ภาษาไทยทั้งหมดจากโฟลเดอร์ที่กำหนด
    """
    documents = []
    
    # วนลูปอ่านทุกไฟล์ PDF ในโฟลเดอร์
    for filename in os.listdir(pdf_directory):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_directory, filename)
            
            # ใช้ PyPDFLoader สำหรับอ่านไฟล์ PDF
            loader = PyPDFLoader(pdf_path)
            pdf_documents = loader.load()
            
            # เพิ่ม metadata เกี่ยวกับที่มาของเอกสาร
            for doc in pdf_documents:
                doc.metadata['source_file'] = filename
            
            documents.extend(pdf_documents)
    
    return documents

In [38]:
def preprocess_thai_text(text):
    """
    Preprocess Thai text using pythainlp
    """
    # Normalize text (standardize characters, remove duplicate spaces)
    normalized_text = normalize(text)
    
    # Word tokenization
    tokens = word_tokenize(normalized_text, engine="newmm")
    
    # Join tokens back with spaces for better chunk splitting
    processed_text = " ".join(tokens)
    
    return processed_text

In [39]:
def create_thai_embeddings(pdf_directory, chunk_size=500):
    """
    สร้าง embeddings จากไฟล์ PDF ภาษาไทย
    """
    # อ่านไฟล์ PDF ทั้งหมด
    documents = []
    for filename in os.listdir(pdf_directory):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_directory, filename)
            loader = PyPDFLoader(pdf_path)
            pdf_documents = loader.load()
            for doc in pdf_documents:
                doc.metadata['source_file'] = filename
            documents.extend(pdf_documents)

    # แบ่ง chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=50,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    
    texts = text_splitter.split_documents(documents)
    
    # สร้าง embeddings
    embedding_function = OllamaEmbeddings(
        model=OLLAMA_EMBED_MODEL,
        base_url=OLLAMA_BASE_URL,
    )
    
    # เก็บใน Chroma - ใช้ embedding_function แทน embeddings
    vectorstore = Chroma.from_documents(
        documents=texts,
        embedding_function=embedding_function,  # แก้จาก embeddings เป็น embedding_function
        collection_name="thai_pdf_docs",
        persist_directory="./chroma_db",
        collection_metadata={
            "language": "thai",
            "chunk_size": chunk_size
        }
    )
    
    return vectorstore


In [40]:

# def create_thai_embeddings(pdf_directory, chunk_size=500):
#     """
#     สร้าง embeddings จากไฟล์ PDF ภาษาไทย
#     """
#     # อ่านไฟล์ PDF ทั้งหมด
#     raw_documents = read_thai_pdfs(pdf_directory)
    
#     # ประมวลผลข้อความแต่ละส่วน
#     for doc in raw_documents:
#         doc.page_content = preprocess_thai_text(doc.page_content)
    
#     # แบ่ง chunks
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=chunk_size,
#         chunk_overlap=50,
#         length_function=len,
#         separators=["\n\n", "\n", " ", ""]
#     )
    
#     texts = text_splitter.split_documents(raw_documents)
    
#     # สร้าง embeddings
#     local_embeddings = OllamaEmbeddings(
#         model=OLLAMA_EMBED_MODEL,
#         base_url=OLLAMA_BASE_URL,
#     )

#     print(local_embeddings)
    
#     # เก็บใน Chroma
#     vectorstore = Chroma.from_documents(
#         documents=texts,
#         embeddings=local_embeddings,
#         collection_name="thai_pdf_docs",
#         persist_directory="./chroma_db",
#         collection_metadata={
#             "language": "thai",
#             "chunk_size": chunk_size,
#             # "source": "pdf"
#         }
#     )
    
#     return vectorstore

In [41]:
# ตัวอย่างการใช้งาน
pdf_directory = "./files"  # โฟลเดอร์ที่เก็บไฟล์ PDF
vectorstore = create_thai_embeddings(pdf_directory)

# ตัวอย่างการค้นหา
# query = "คำค้นหาภาษาไทย"
# results = vectorstore.similarity_search(query)
# for doc in results:
#     print(f"Source: {doc.metadata['source_file']}, Page: {doc.metadata['page']}")
#     print(doc.page_content)
#     print("-" * 50)

TypeError: langchain_community.vectorstores.chroma.Chroma() got multiple values for keyword argument 'embedding_function'